In [1]:
# -*- coding: utf-8 -*-
# policy_search.py
# ============================
# 여성 정책 큐레이션: 순수 파이썬 버전 (UI/HTML 제거)
# - 임베딩/FAISS 인덱스 디스크 캐시 지원
# - input가 비어도/문장이어도 안전
# ============================

import os, re, time, json, argparse, hashlib, io
from datetime import datetime, date
from typing import List, Optional, Literal, Tuple

import numpy as np
import pandas as pd
from dateutil import parser
from sentence_transformers import SentenceTransformer
import faiss, torch

# ============================
# 0) 경로 & 공통
# ============================
FILE_PATH = os.getenv("POLICY_XLSX", "여성맞춤정책_요약_2차_결과_정리.xlsx")
assert os.path.exists(FILE_PATH), f"엑셀 파일이 존재하지 않음: {FILE_PATH}"

CACHE_DIR = os.getenv("POLICY_CACHE_DIR", ".policy_cache")
os.makedirs(CACHE_DIR, exist_ok=True)

# ============================
# 1) 데이터 로드 & 전처리
# ============================
df_raw = pd.read_excel(FILE_PATH)
df = df_raw.dropna(subset=["제목"]).copy()

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    t = re.sub(r"(?m)^\s*([\-–—\u2010-\u2015\u2212\u2043\u2022\u25CB\u25CF\u25AA\u25A0\u30FB]|[0-9]+[.)])\s+", " ", t)
    t = re.sub(r"[\u2460-\u2473\u3251-\u325F\u32B1-\u32BF]", " ", t)  # 원형 숫자
    t = re.sub(r"[※＊*]+", " ", t)
    t = re.sub(r"\s{2,}", " ", t).strip()
    return t

def clean_field_value(val: str, field_name: str) -> str:
    if not isinstance(val, str): return ""
    v = re.sub(rf"^\s*{re.escape(field_name)}\s*[:\-–—]?\s*", "", val.strip())
    return normalize_text(v)

def unify_text_natural(row: pd.Series) -> str:
    title   = normalize_text(row.get("제목",""))
    region  = normalize_text(row.get("지역",""))
    target  = clean_field_value(row.get("지원대상",""), "지원대상")
    content = clean_field_value(row.get("지원내용",""), "지원내용")
    parts = []
    if title:   parts.append(f"정책명은 {title}")
    if region:  parts.append(f"지역은 {region}")
    if target:  parts.append(f"지원대상은 {target}")
    if content: parts.append(f"지원내용은 {content}")
    if not parts: 
        return ""
    if len(parts) == 1: 
        return parts[0] + "이다."
    if len(parts) == 2: 
        return parts[0] + "이고, " + parts[1] + "이다."
    return "이고, ".join(parts[:-1]) + "이며, " + parts[-1] + "이다."

TEXT_COL = "검색본문_nat"
df[TEXT_COL] = df.apply(unify_text_natural, axis=1)

# ============================
# 2) 임베딩 & 인덱스 (KURE-v1 + FAISS) + 캐시
# ============================
MODEL_NAME = "nlpai-lab/KURE-v1"
device = (
    "cuda" if torch.cuda.is_available() else
    ("mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu")
)
print(f"✅ 모델 로드: {MODEL_NAME} (device={device})")
model = SentenceTransformer(MODEL_NAME, device=device)

def _corpus_digest(corpus: List[str]) -> str:
    """모델명+행수+텍스트 해시 생성(변경 감지)."""
    h = hashlib.sha256()
    h.update(MODEL_NAME.encode("utf-8"))
    h.update(str(len(corpus)).encode("utf-8"))
    # 메모리 과다 방지: 줄마다 업데이트 (대용량 안전)
    for line in corpus:
        if not isinstance(line, str):
            line = "" if pd.isna(line) else str(line)
        # 줄바꿈 구분자 포함
        h.update(line.encode("utf-8", errors="ignore"))
        h.update(b"\n")
    return h.hexdigest()

# === 위쪽 import 근처에 re 이미 있음 ===

def _safe_part(s: str) -> str:
    """파일명 안전 문자열: ASCII만 남기고 나머지는 '_'로 대체. 비면 'corpus'."""
    if not isinstance(s, str):
        s = str(s)
    # 1) ASCII만 남기기
    s_ascii = s.encode("ascii", "ignore").decode("ascii")
    # 2) 안전하지 않은 문자 -> '_'
    s_ascii = re.sub(r"[^A-Za-z0-9._-]+", "_", s_ascii)
    s_ascii = s_ascii.strip("_.")
    return s_ascii or "corpus"

def _cache_paths(corpus: List[str]) -> Tuple[str, str, str]:
    base   = os.path.splitext(os.path.basename(FILE_PATH))[0]
    digest = _corpus_digest(corpus)[:16]
    base_s  = _safe_part(base)                          # ← 한글 제거
    model_s = _safe_part(MODEL_NAME.replace("/", "_"))  # ← 슬래시 등 제거

    key = f"{base_s}.{len(corpus)}.{model_s}.{digest}"

    # Windows 경로 길이 대비: 키가 너무 길면 줄이기
    if len(key) > 120:
        key = f"{base_s[:30]}.{len(corpus)}.{model_s[:30]}.{digest}"

    faiss_path = os.path.join(CACHE_DIR, f"{key}.faiss")
    npy_path   = os.path.join(CACHE_DIR, f"{key}.npy")
    meta_path  = os.path.join(CACHE_DIR, f"{key}.json")
    return faiss_path, npy_path, meta_path

def _save_cache(index: faiss.Index, embeddings: np.ndarray, meta_path: str, faiss_path: str, npy_path: str):
    faiss.write_index(index, faiss_path)
    np.save(npy_path, embeddings)
    meta = {
        "model": MODEL_NAME,
        "ntotal": int(index.ntotal),
        "embedding_dim": int(embeddings.shape[1]),
        "saved_at": datetime.now().isoformat(timespec="seconds"),
        "file": os.path.basename(FILE_PATH),
        "text_col": TEXT_COL,
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

def _load_cache(faiss_path: str, npy_path: str) -> Tuple[faiss.Index, np.ndarray]:
    index = faiss.read_index(faiss_path)
    embeddings = np.load(npy_path)
    return index, embeddings

def _build_or_load_index(corpus: List[str], force_rebuild: bool = False) -> Tuple[faiss.Index, np.ndarray]:
    faiss_path, npy_path, meta_path = _cache_paths(corpus)
    if (not force_rebuild) and os.path.exists(faiss_path) and os.path.exists(npy_path):
        try:
            index, embeddings = _load_cache(faiss_path, npy_path)
            if index.ntotal != len(corpus):
                raise RuntimeError("캐시 ntotal 불일치")
            print(f"⚡ 캐시 로드 완료: {faiss_path}")
            return index, embeddings
        except Exception as e:
            print(f"캐시 로드 실패, 재생성합니다. 이유: {e}")

    print("✅ 정책 임베딩 생성...")
    t0 = time.time()
    embeddings = model.encode(
        corpus, convert_to_numpy=True, batch_size=64, show_progress_bar=True
    ).astype("float32")
    faiss.normalize_L2(embeddings)                 # 코사인 유사도 = L2 정규화 + Inner Product
    index = faiss.IndexFlatIP(embeddings.shape[1]) # 내적 기반 인덱스
    index.add(embeddings)
    print(f"✅ 임베딩 완료: {time.time()-t0:.2f}s, 벡터수={len(embeddings)}")

    # 캐시에 저장
    try:
        _save_cache(index, embeddings, meta_path, faiss_path, npy_path)
        print(f"💾 캐시 저장: {faiss_path}")
    except Exception as e:
        print(f"캐시 저장 실패(무시): {e}")

    return index, embeddings

# 캐시 재빌드 여부(환경변수로도 제어)
FORCE_REBUILD = os.getenv("POLICY_REBUILD", "0") in ("1","true","True","YES","yes")
corpus = df[TEXT_COL].fillna("").tolist()
index, _embeddings = _build_or_load_index(corpus, force_rebuild=FORCE_REBUILD)

# ============================
# 3) 1차 필터(지역/생년/카테고리/지원형태)
# ============================
REGION_ALIAS = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기": "경기도",
    "강원": "강원특별자치도|강원도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전북특별자치도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도|제주도",
}

def expand_region_tokens(user_region: str) -> List[str]:
    if not user_region: return []
    raw_tokens = [tok for tok in re.split(r"[,/ ]+", user_region) if tok]
    out = set()
    for t in raw_tokens:
        out.add(t)
        if t in REGION_ALIAS:
            out.update(REGION_ALIAS[t].split("|"))
        for short,longs in REGION_ALIAS.items():
            if t in longs.split("|"): 
                out.add(short)
    return list(out)

def parse_birthdate(s: str):
    if not s: return None
    for fmt in ("%Y-%m-%d","%Y.%m.%d","%Y/%m/%d"):
        try: 
            return datetime.strptime(s.strip(), fmt).date()
        except: 
            pass
    try: 
        return parser.parse(s.strip()).date()
    except: 
        return None

def calc_age_years_precise(birth, ref=None):
    if not birth: return None
    if ref is None: ref = date.today()
    return (ref - birth).days/365.2425

KW_RANGES = [
    (re.compile(r"가임기\s*여성|가임기여성"), (15,49)),
    (re.compile(r"청소년"), (13,20)),
    (re.compile(r"아동"), (0,12.999)),
    (re.compile(r"어린이"), (3,13)),
    (re.compile(r"노인|고령자|어르신"), (65,float("inf"))),
]

def _norm_min(s): 
    if not isinstance(s,str): return ""
    return re.sub(r"\s{2,}"," ", s.replace("\r"," ").replace("\n"," ").replace("\t"," ")).strip()

def extract_age_constraints(t):
    t = _norm_min(t or ""); cons=[]
    for m in re.finditer(r"(?:만\s*)?(\d+)\s*세\s*이상\s*[~\-–]\s*(?:만\s*)?(\d+)\s*세\s*미만", t):
        lo,hi=int(m[1]),int(m[2]); cons.append((lo,hi,True,False))
    for m in re.finditer(r"만\s*(\d+)\s*세\s*이상\s*만\s*(\d+)\s*세\s*이하", t):
        lo,hi=int(m[1]),int(m[2]); cons.append((lo,hi,True,True))
    for m in re.finditer(r"(?:만\s*)?(\d+)\s*세\s*[~\-–]\s*(?:만\s*)?(\d+)\s*세", t):
        lo,hi=int(m[1]),int(m[2]); cons.append((lo,hi,True,True))
    for m in re.finditer(r"(?:만\s*)?(\d+)\s*[~\-–]\s*(?:만\s*)?(\d+)\s*세", t):
        lo,hi=int(m[1]),int(m[2]); cons.append((lo,hi,True,True))
    for m in re.finditer(r"(?:만\s*)?(\d+)\s*세\s*(이상|이하|미만|초과|세이상|세이하|세미만)", t):
        v=int(m[1]); typ=m[2]
        if typ in ("이상","세이상"): cons.append((v,None,True,None))
        elif typ in ("이하","세이하"): cons.append((None,v,None,True))
        elif typ in ("미만","세미만"): cons.append((None,v,None,False))
        elif typ=="초과": cons.append((v,None,False,None))
    for m in re.finditer(r"생후\s*(\d+)\s*개월\s*[~\-–]\s*(?:만\s*)?(\d+)\s*세", t):
        lo=float(m[1])/12.0; hi=int(m[2]); cons.append((lo,hi,True,True))
    return cons

def extract_kw_constraints(t):
    t=_norm_min(t or ""); out=[]
    for pat,(lo,hi) in KW_RANGES:
        if pat.search(t): out.append((float(lo), float(hi), True, True))
    return out

def age_ok(text, birth_str):
    if not birth_str: return True
    birth = parse_birthdate(birth_str)
    if not birth: return True
    age = calc_age_years_precise(birth)
    t = normalize_text(text or "")
    cons = extract_age_constraints(t)
    kwc  = extract_kw_constraints(t)
    if cons:
        los=[c[0] for c in cons if c[0] is not None]
        his=[c[1] for c in cons if c[1] is not None]
        lo=max(los) if los else None
        hi=min(his) if his else None
        def inwin(a, lo, hi):
            if lo is not None and a<lo: return False
            if hi is not None and a>hi: return False
            return True
        if lo is None and hi is None and kwc:
            ok=False
            for k in kwc:
                klo,khi = k[0],k[1]
                lo2 = max(klo, lo) if lo is not None else klo
                hi2 = min(khi, hi) if hi is not None else khi
                if lo2<=hi2 and inwin(age, lo2, hi2): ok=True; break
            return ok
        return inwin(age, lo, hi)
    if kwc:
        for k in kwc:
            klo,khi=k[0],k[1]
            if klo<=age<=khi: return True
        return False
    return True  # 신호 없으면 배제하지 않음

def split_labels(s: str) -> List[str]:
    if not isinstance(s, str): return []
    return [x.strip() for x in re.split(r"[;,/|]", s) if x.strip()]

CATEGORY_KEYS = [
    "1인가구","한부모","임신/출산/육아","초/중/고등학생","대학(원)생","근로자/직장인",
    "구직/취업","창업/자영업","고령자","신혼부부","다문화","농/어업인","취약계층"
]
SUPPORT_KEYS = [
    "직접금전지원","감면할인지원","이용권","교육지원","의료지원","주거지원","교통지원",
    "생계지원","상담지원","물품지원","문화예술","서비스","보험료지원","법률지원"
]

def contains_any_label(cell: str, chosen: List[str]) -> bool:
    if not chosen: return True
    row_tags = set(split_labels(cell))
    return any(tag in row_tags for tag in chosen)

def build_stage1_mask(df_in: pd.DataFrame,
                      categories: Optional[List[str]]=None,
                      supports: Optional[List[str]]=None,
                      region: Optional[str]=None,
                      birthdate: Optional[str]=None) -> np.ndarray:
    n = len(df_in)
    mask = np.ones(n, dtype=bool)

    if categories and "category_label" in df_in.columns:
        mask &= df_in["category_label"].astype(str).apply(lambda x: contains_any_label(x, categories)).to_numpy()
    if supports and "support_label" in df_in.columns:
        mask &= df_in["support_label"].astype(str).apply(lambda x: contains_any_label(x, supports)).to_numpy()

    if region and "지역" in df_in.columns:
        toks = expand_region_tokens(region)
        pat  = "|".join(re.escape(t) for t in toks) if toks else ""
        rs   = df_in["지역"].astype(str)
        rmask = rs.str.contains("전국", na=False) | (rs.str.contains(pat, na=False) if pat else False)
        mask &= rmask.to_numpy()

    if birthdate and "지원대상" in df_in.columns:
        mask &= df_in["지원대상"].apply(lambda x: age_ok(x, birthdate)).to_numpy()

    return mask

def filter_by_user_inputs(df_in: pd.DataFrame,
                          region: Optional[str],
                          dob: Optional[str],
                          categories: Optional[List[str]],
                          supports: Optional[List[str]]) -> pd.DataFrame:
    mask = build_stage1_mask(
        df_in, categories=categories, supports=supports, region=region, birthdate=dob
    )
    return df_in[mask].copy()

# ============================
# 4) 시맨틱 검색 (FAISS)
# ============================
def _faiss_search(query: str, top_k: int):
    total = int(index.ntotal)
    safe_k = max(1, min(int(top_k)*5, total if total > 0 else 1))
    q = model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q)
    D, I = index.search(q, k=safe_k)
    return D[0], I[0]

def semantic_search(query: str, top_k: int = 10, out: Literal["dataframe","json","csv"]="dataframe"):
    """자연어 검색. 상위 k개 반환. 쿼리 비면 빈 결과 반환."""
    if not query or not str(query).strip():
        if out == "dataframe":
            return pd.DataFrame()
        return "[]"
    D, I = _faiss_search(str(query).strip(), int(top_k))
    I, D = I[:int(top_k)], D[:int(top_k)]
    rows = df.iloc[I].reset_index(drop=True).copy()
    rows.insert(0, "score", np.round(D, 4))
    return _format_output(rows, out)

# ============================
# 5) 추천 (필터 + 정렬)
# ============================
def _last_date_from_period(s: str):
    """'신청기간' 컬럼 문자열에서 마지막 날짜를 뽑아 정렬 키로 사용."""
    if not isinstance(s,str) or not s.strip(): 
        return pd.NaT
    s2 = s.replace("~","-").replace("–","-").replace("—","-")
    parts = re.findall(r"\d{4}[./-]\d{1,2}[./-]\d{1,2}", s2)
    try:
        return parser.parse(parts[-1]) if parts else pd.NaT
    except:
        return pd.NaT

def recommend(region: Optional[str]="전국",
              dob: Optional[str]=None,
              categories: Optional[List[str]]=None,
              supports: Optional[List[str]]=None,
              limit_n: int = 20,
              out: Literal["dataframe","json","csv"]="dataframe"):
    """사용자 조건 기반 추천. 상위 N개 반환."""
    filtered = filter_by_user_inputs(df, region, dob, categories or [], supports or [])
    tmp = filtered.copy()
    if "신청기간" in tmp.columns:
        tmp["_end"] = tmp["신청기간"].apply(_last_date_from_period)
        tmp = tmp.sort_values(by=["_end","제목"], ascending=[True,True]).drop(columns=["_end"])
    tmp = tmp.head(int(limit_n)).reset_index(drop=True)
    return _format_output(tmp, out)

# ============================
# 6) 통합 진입점: input(문장) or 빈 문자열
# ============================
def find_policies(input: str = "",
                  topk: int = 20,
                  region: Optional[str] = "전국",
                  dob: Optional[str] = None,
                  categories: Optional[List[str]] = None,
                  supports: Optional[List[str]] = None,
                  out: Literal["dataframe","json","csv"]="dataframe"):
    """
    input이 비면 → 추천; input이 있으면 → 시맨틱 검색(+선택적 필터 적용)
    필터는 있으면 적용, 없으면 무시.
    """
    if not input or not str(input).strip():
        return recommend(region=region, dob=dob, categories=categories, supports=supports,
                         limit_n=topk, out=out)

    D, I = _faiss_search(str(input).strip(), int(topk))
    mask = build_stage1_mask(df, categories or [], supports or [], region, dob)
    picked = [(i, float(d)) for i, d in zip(I, D) if mask[i]]
    if not picked:
        I2, D2 = I[:int(topk)], D[:int(topk)]
        rows = df.iloc[I2].reset_index(drop=True).copy()
        rows.insert(0, "score", np.round(D2[:len(rows)], 4))
        return _format_output(rows, out)

    I_f = [i for i, _ in picked][:int(topk)]
    D_f = [d for _, d in picked][:int(topk)]
    rows = df.iloc[I_f].reset_index(drop=True).copy()
    rows.insert(0, "score", np.round(D_f, 4))
    return _format_output(rows, out)

# ============================
# 7) 출력 포맷터
# ============================
def _format_output(df_out: pd.DataFrame, out: Literal["dataframe","json","csv"]):
    if out == "dataframe":
        return df_out
    if out == "json":
        cols = [c for c in ["score","제목","지역","category_label","support_label","지원형태",
                            "신청기간","신청방법","접수기관","지원대상","지원내용","문의처","기타","detail_url"]
                if c in df_out.columns]
        return json.dumps(df_out[cols].to_dict(orient="records"), ensure_ascii=False, indent=2)
    if out == "csv":
        return df_out.to_csv(index=False)
    raise ValueError("out must be one of {'dataframe','json','csv'}")

# ============================
# 8) CLI (find / recommend / search)
# ============================
def _parse_list(s: Optional[str]) -> List[str]:
    if not s: return []
    return [x.strip() for x in re.split(r"[;,/|,]", s) if x.strip()]

def main(argv: Optional[List[str]] = None):
    import sys
    ap = argparse.ArgumentParser(description="여성 정책 큐레이션 (UI/HTML 제거)")
    sub = ap.add_subparsers(dest="cmd", required=True)

    # 통합 커맨드: input이 비면 추천, 있으면 검색
    ap_find = sub.add_parser("find", help="input이 비면 추천, 있으면 검색")
    ap_find.add_argument("--input", type=str, default="", help="자연어 쿼리(빈 문자열이면 추천 모드)")
    ap_find.add_argument("--topk", type=int, default=20)
    ap_find.add_argument("--region", type=str, default="전국")
    ap_find.add_argument("--dob", type=str, default=None)
    ap_find.add_argument("--categories", type=str, default=None, help="쉼표구분")
    ap_find.add_argument("--supports", type=str, default=None, help="쉼표구분")
    ap_find.add_argument("--out", type=str, choices=["dataframe","json","csv"], default="json")
    ap_find.add_argument("--rebuild", action="store_true", help="임베딩/인덱스 캐시 강제 재생성")

    # 개별 커맨드
    ap_rec = sub.add_parser("recommend", help="조건 기반 추천")
    ap_rec.add_argument("--region", type=str, default="전국")
    ap_rec.add_argument("--dob", type=str, default=None)
    ap_rec.add_argument("--categories", type=str, default=None, help="쉼표구분 예) 대학(원)생,근로자/직장인")
    ap_rec.add_argument("--supports", type=str, default=None, help="쉼표구분 예) 교육지원,주거지원")
    ap_rec.add_argument("--limit", type=int, default=20)
    ap_rec.add_argument("--out", type=str, choices=["dataframe","json","csv"], default="json")
    ap_rec.add_argument("--rebuild", action="store_true", help="임베딩/인덱스 캐시 강제 재생성")

    ap_srch = sub.add_parser("search", help="자연어 시맨틱 검색")
    ap_srch.add_argument("--query", type=str, required=True)
    ap_srch.add_argument("--topk", type=int, default=10)
    ap_srch.add_argument("--out", type=str, choices=["dataframe","json","csv"], default="json")
    ap_srch.add_argument("--rebuild", action="store_true", help="임베딩/인덱스 캐시 강제 재생성")

    if argv is None:
        argv = sys.argv[1:]

    args = ap.parse_args(argv)

    # 필요 시 강제 재빌드
    if getattr(args, "rebuild", False):
        global index, _embeddings
        print("🔄 --rebuild 지정됨: 캐시 무시하고 재생성합니다.")
        # 재해시/재생성
        new_index, new_emb = _build_or_load_index(corpus, force_rebuild=True)
        index, _embeddings = new_index, new_emb

    if args.cmd == "find":
        res = find_policies(
            input=args.input,
            topk=args.topk,
            region=args.region,
            dob=args.dob,
            categories=_parse_list(args.categories),
            supports=_parse_list(args.supports),
            out=args.out
        )
        if isinstance(res, pd.DataFrame):
            print(res.to_csv(index=False, sep="\t"))
        else:
            print(res)

    elif args.cmd == "recommend":
        res = recommend(
            region=args.region,
            dob=args.dob,
            categories=_parse_list(args.categories),
            supports=_parse_list(args.supports),
            limit_n=args.limit,
            out=args.out
        )
        if isinstance(res, pd.DataFrame):
            print(res.to_csv(index=False, sep="\t"))
        else:
            print(res)

    elif args.cmd == "search":
        res = semantic_search(
            query=args.query, top_k=args.topk, out=args.out
        )
        if isinstance(res, pd.DataFrame):
            print(res.to_csv(index=False, sep="\t"))
        else:
            print(res)

if __name__ == "__main__":
    # 터미널에서만 CLI 실행 (노트북/인터프리터에서는 함수만 사용)
    import sys
    in_ipy = ("ipykernel" in sys.modules) or ("IPython" in sys.modules)
    if not in_ipy:
        main()


c:\Users\hyunj\anaconda3\envs\env1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 모델 로드: nlpai-lab/KURE-v1 (device=cpu)
⚡ 캐시 로드 완료: .policy_cache\2.1292.nlpai-lab_KURE-v1.ad5f6c49b0642012.faiss


In [4]:
# ============================
# 1) 빈 입력 → 추천 (전국 기준 상위 20)
df_rec = find_policies(input="45세", topk=20, out="dataframe")

# 2) 자연어 쿼리 → 검색 (필터 없음)
js = find_policies(input="서울에서 지원하는 임산부 정책", topk=15, out="dataframe")

# 2) 자연어 쿼리 → 검색 (필터 없음)
js2 = find_policies(input="대구에서 임산부 부부 백일해 및 임신준비 여성 풍진 예방접종", topk=15, out="dataframe")


# 3) 자연어 + 필터 동시 (지역/카테고리/생년 등)
csv_text = find_policies(
    input="임산부 교통 지원",
    region="서울",
    categories=["임신/출산/육아"],
    dob="1997-05-20",
    topk=30,
    out="dataframe"
)

In [5]:
df_rec.head(5)  # 추천 결과 상위 5개

,score,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,지원내용,문의처,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약,검색본문_nat
0,0.4443,여성,전국,이야기 할머니 현장 활동 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/9990...,2025년 1월 15일 ~ 2025년 2월 14일,"○ 연1회 선발, 우편, 온라인, 방문으로 원서접수, 서류 및 면접전형\n○ 아름다...",한국국학진흥원,현금,지원대상\n○ 만 56세에서 74세의 여성 어르신\n선정기준\n○ 선발전형에 합격하...,지원내용\n○ 유아교육기관에서 유아들에게 이야기 들려주는 활동비 1회 4만원 지원 ...,한국국학진흥원 이야기할머니사업단 ( ☎054-851-0861 ),NaN,고령자,"직접금전지원,교육지원",(1) 만 56세에서 74세의 여성 어르신\n(2) 선정기준\n - 선발전형에 합격...,만 56세에서 74세의 여성 어르신,(1) 유아교육기관에서 유아들에게 이야기 들려주는 활동비 1회 4만원 지원 ( 연 ...,(1) 유아교육기관에서 유아들에게 이야기 들려주는 활동비 1회 4만원 지원 ( 연 ...,"정책명은 이야기 할머니 현장 활동 지원이고, 지역은 전국이고, 지원대상은 만 56세..."
1,0.4161,한부모,전국,노인실명예방사업,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1352...,상시신청,"○ ( 대상자 )\n - 수술 받기 전, 보건소에 서류 접수\n* 제출서류 ( 1개...",보건소,서비스 ( 의료 ),지원대상\n○ 안 검진: 검진신청지역 만 60세 이상 노인 ( 저소득층 노인 )\n...,"지원내용\n○ 지원내용\n - 안 검진: 시력검사, 굴절검사, 안압검사, 세극등 현...",보건복지상담센터 ( ☎129 )\n한국실명예방재단 ( ☎02-718-1102 ),NaN,"한부모,고령자,취약계층","교육지원,의료지원,상담지원,서비스",(1) 안 검진: 검진신청지역 만 60세 이상 노인 ( 저소득층 노인 )\n(2) ...,만 60세 이상으로서 저소득계층,"(1) 세부사항\n - 안 검진: 시력검사, 굴절검사, 안압검사, 세극등 현미경검사...","지원 제외: 질병 치료비, 간병비, 상급병실료 등 비급여, 한국실명예방재단의 승인 ...","정책명은 노인실명예방사업이고, 지역은 전국이고, 지원대상은 안 검진: 검진신청지역 ..."
2,0.4149,한부모,전국,노인 무릎인공관절 수술 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1352...,상시신청,"○ ( 지원자 ) 수술받기 전, 보건소에 서류 접수\n○ ( 보건소 ) 노인의료나눔...",보건소,서비스 ( 의료 ),지원대상\n○ ( 연령 ) 만 60세 이상 \n\n○ ( 대상 질환 ) 건강보험급여...,지원내용\n○ ( 수술비 지원액 ) 한쪽 무릎 기준 120만원 한도 실비 지원\n\...,노인의료나눔재단 ( ☎1661-6595 ),NaN,"한부모,고령자,취약계층","직접금전지원,의료지원,서비스",(1) 연령: 만 60세 이상\n(2) 대상 질환:건강보험급여 '인공관절 치환술 (...,건강보험급여 '인공관절 치환술 ( 무릎관절 ) ' 인정기준에 준하는 질환자 및 만 ...,(1) 수술비:지원액 한쪽 무릎 기준 120만원 한도 실비 지원\n(2) 지원범위:...,수술비 지원 등,"정책명은 노인 무릎인공관절 수술 지원이고, 지역은 전국이고, 지원대상은 ( 연령 )..."
3,0.4077,한부모,전국,청년내일저축계좌,복지로 ( bokjiro.go.kr ),복지로 홈페이지 확인,읍·면·동 행정복지센터 또는 복지로 홈페이지에서 온라인 신청,읍·면·동 행정복지센터 또는 복지로 홈페이지,현금,"신청 당시 19∼34세 ( 단, 수급자·차상위자는 15∼39세 ) 의 기준 중위소득...",본인 저축금 ( 월 10만 원∼50만 원 ) 적립 시 정부매칭금 월10만원 지원\n...,읍·면·동 행정복지센터 / 주민센터\n자산형성지원콜센터 ( ☎1522-3690 ),NaN,"창업/자영업,취약계층",직접금전지원,"(1) 신청 당시 19∼34세 ( 단, 수급자·차상위자는 15∼39세 ) 의 기준 ...","신청 당시 19∼34세 ( 단, 수급자·차상위자는 15∼39세 ) 의 기준 중위소득...",(1) 본인 저축금 ( 월 10만 원∼50만 원 ) 적립 시 정부매칭금 월10만원 ...,본인 저축금 ( 월 10만 원∼50만 원 ) 적립 시 정부매칭금 월10만원 지원,"정책명은 청년내일저축계좌이고, 지역은 전국이고, 지원대상은 신청 당시 19∼34세 ..."
4,0.4077,한부모,전국,국민취업지원제도,고용24 ( www.work24.go.kr ),상시 신청,고용24 온라인 신청 또는 고용센터 방문,"관할 고용센터 또는 온라인 ( 워크넷, 고용24 )","직업훈련, 일경험, 복지서비스 연계, 취업알선 등","근로능력과 구직의사가 있는 15~69세 이하의 저소득층, 미혼모, 한부모 등",참여자의 소득과 재산 등에 따라 두 유형으로 지원\n - ( 공통 ) 1년간 취업에...,고용노동부 상담센터 ( ☎1350 ),NaN,"한부모,구직/취업,취약계층","직접금전지원,서비스","(1) 근로능력과 구직의사가 있는 15~69세 이하의 저소득층, 미혼모, 한부모 등","(1) 근로능력과 구직의사가 있는 15~69세 이하의 저소득층, 미혼모, 한부모 등",(1) 참여자의 소득과 재산 등에 따라 두 유형으로 지원\n - ( 공통 ) 1년간...,참여자의 소득과 재산 등에 따라 두 유형으로 지원,"정책명은 국민취업지원제도이고, 지역은 전국이고, 지원대상은 근로능력과 구직의사가 있..."


In [7]:
js.head(5)  # 검색 결과 상위 5개

,score,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,지원내용,문의처,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약,검색본문_nat
0,0.6639,한부모,전국,청소년산모 임신·출산 의료비 지원,https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...,임신 확인 후 상시,사회서비스 전자바우처 신청,사회서비스 바우처센터,추가 의료비 지원 ( 1회당 120만원 한도 ),임신확인서로 임신이 확인된 19세 이하의 청소년 산모,● 임신 1회당 의료비 120만 원 이내 추가 지원,"사회서비스 전자바우처\n( ☎1566-3232 (단축번호4 ) , www.socia...",NaN,"임신/출산/육아,초/중/고등학생",의료지원,(1) 임신확인서로 임신이 확인된 19세 이하의 청소년 산모,(1) 임신확인서로 임신이 확인된 19세 이하의 청소년 산모,(1) 임신 1회당 의료비 120만 원 이내 추가 지원,(1) 임신 1회당 의료비 120만 원 이내 추가 지원,"정책명은 청소년산모 임신·출산 의료비 지원이고, 지역은 전국이고, 지원대상은 임신확..."
1,0.6575,임산부,전국,표준 모자보건수첩 제공,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/GMW0...,접수기관별상이,"보건소 또는 의료기관 ( 산부인과, 소아청소년과 ) , 읍면동 주민센터 ( 점자수첩...",보건소,현물,지원대상\n○ 임신부 또는 출생 사실 확인된 영유아\n선정기준\n○ 보건소 등록 임산부,지원내용\n○ 임산부수첩 및 아기수첩 제공,해당지역 보건소 ( ☎- ),NaN,임신/출산/육아,물품지원,(1) 임신부 또는 출생 사실 확인된 영유아\n(2) 선정기준\n - 보건소 등록 임산부,임신부 또는 출생 사실 확인된 영유아,(1) 임산부수첩 및 아기수첩 제공,(1) 임산부수첩 및 아기수첩 제공,"정책명은 표준 모자보건수첩 제공이고, 지역은 전국이고, 지원대상은 임신부 또는 출생..."
2,0.6409,임산부,전국,청소년산모 임신·출산 의료비 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/SD00...,임신확인서 상 임신확인일 기준으로 만 19세까지 신청 가능,○ 요양기관에서 '청소년 산모 임신출산 의료비 지원 신청 및 임신확인서' 발급\n\...,사회서비스 전자바우처 온라인 신청,서비스 ( 의료 ),지원대상\n○ 만 19세 이하 산모\n선정기준\n○ 지원 대상 : 임신부\n\n○ ...,지원내용\n○ 임신 1회당 120만 원 범위 내,보건복지상담센터 ( ☎129 ),NaN,"임신/출산/육아,초/중/고등학생","의료지원,서비스",(1) 만 19세 이하 산모\n(2) 임신부\n(3) 연령 기준 : 만 19세 이하,(1) 만 19세 이하 산모\n(2) 임신부\n(3) 연령 기준 : 만 19세 이하,(1) 임신 1회당 120만 원 범위 내,(1) 임신 1회당 120만 원 범위 내,"정책명은 청소년산모 임신·출산 의료비 지원이고, 지역은 전국이고, 지원대상은 만 1..."


In [10]:
js2.head(5)  # 검색 결과 상위 5개

,score,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,지원내용,문의처,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약,검색본문_nat
0,0.7967,임산부,대구광역시/남구,임산부 부부 백일해 및 임신준비 여성 풍진 예방접종,https://www.nam.daegu.kr/health/index.do?menu_...,연중 상시,남구보건소 방문신청,대구광역시 남구보건소,"백일해 ( Tdap ) : 임산부-매 임신시, 배우자-1회\n풍진 ( MMR ) : 1회",접종일 현재 대구 남구에 주민등록을 둔 자 ( 여성기준 )\n백일해: 임신 27-3...,백일해 ( Tdap ) 및 풍진 ( MMR ) 무료 예방접종,"보건소 보건행정과 ( ☎664-3688,3692 )",NaN,"임신/출산/육아,신혼부부,다문화",의료지원,(1) 접종일 현재 대구 남구에 주민등록을 둔 자 ( 여성기준 )\n(2) 백일해:...,대구 남구에 주민등록을 둔 자 등,(1) 백일해 ( Tdap ) 및 풍진 ( MMR ) 무료 예방접종,(1) 백일해 ( Tdap ) 및 풍진 ( MMR ) 무료 예방접종,"정책명은 임산부 부부 백일해 및 임신준비 여성 풍진 예방접종이고, 지역은 대구광역시..."
1,0.6241,임산부,대구광역시/동구,모자보건사업운영 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3420...,상시신청,○ 방문 신청\n - 보건소 방문: 산모 주소지 관할 보건소에 방문 신청,보건소,서비스 ( 의료 ),지원대상\n○ 임신을 원하는 예비 ( 법적 ) 부부\n\n○ 임신 24~28주 임산부,"지원내용\n○ 임신을 원하는 예비 ( 법적 ) 부부 성병검사, B형간염 항원·항체검...",동구청 건강증진과 ( ☎053-662-3236 ),NaN,"임신/출산/육아,신혼부부","의료지원,서비스",(1) 임신을 원하는 예비 ( 법적 ) 부부\n(2) 임신 24~28주 임산부,(1) 임신을 원하는 예비 ( 법적 ) 부부\n(2) 임신 24~28주 임산부,"(1) 임신을 원하는 예비 ( 법적 ) 부부 성병검사, B형간염 항원·항체검사, 풍...","(1) 임신을 원하는 예비 ( 법적 ) 부부 성병검사, B형간염 항원·항체검사, 풍...","정책명은 모자보건사업운영 지원이고, 지역은 대구광역시/동구이고, 지원대상은 임신을 ..."
2,0.6226,임산부,울산광역시/남구,신혼 ( 임신 ) 부부 백일해 예방접종 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3700...,2024.02.19~2025.12.31,"신청방법\n○ 방문, 온라인 ( 온라인 신청일 기준 7일 후 접종 가능 )\n○ 방...",울산광역시 남구 보건소,서비스 ( 의료 ),지원대상\n○ 주민등록등본상 주소지가 울산 '남구'인 신혼부부\n - 24.1.1....,지원내용\n○ 신혼 및 임신부부 대상 백일해 예방접종 백신 무료접종 지원\n - 2...,보건소 건강행복과 ( ☎052-226-2431 ),NaN,"임신/출산/육아,신혼부부","의료지원,서비스",(1) 주민등록등본상 주소지가 울산 '남구'인 신혼부부\n - 24.1.1. 이후 ...,주민등록등본상 주소지가 울산 '남구'인 신혼부부 임산부와 배우자 중 울산 남구의 임...,(1) 신혼 및 임신부부 대상 백일해 예방접종 백신 무료접종 지원\n - 24.1....,신혼 및 임신부부를 대상으로 백일해 예방접종 백신 무료접종을 지원한다.,"정책명은 신혼 ( 임신 ) 부부 백일해 예방접종 지원이고, 지역은 울산광역시/남구이..."
3,0.6141,임산부,대구광역시/서구,예비부부 ( 부모 ) 무료 건강검진 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3430...,상시신청,○ 방문 신청\n - 보건소 : 관할 보건소 모자보건실 방문,보건소,서비스 ( 의료 ),지원대상\n○ 관내 예비부부 ( 부모 ),지원내용\n○ 무료 건강검진 ( 10종 ) 실시 및 엽산제 지원 ( 여성 )\n B...,모자보건실 ( ☎053-663-3166 )\n모자보건실 ( ☎053-663-3171 ),NaN,신혼부부,"의료지원,서비스",(1) 관내 예비부부 ( 부모 ),(1) 관내 예비부부 ( 부모 ),(1) 무료 건강검진 ( 10종 ) 실시 및 엽산제 지원 ( 여성 )\n(2) B형...,(1) 무료 건강검진 ( 10종 ) 실시 및 엽산제 지원 ( 여성 )\n(2) B형...,"정책명은 예비부부 ( 부모 ) 무료 건강검진 지원이고, 지역은 대구광역시/서구이고,..."
4,0.5992,여성,전라남도/영암군,예비맘 풍진 예방접종지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/4940...,상시신청,○ 구비서류 지참 후 방문 신청,영암군 보건소 및 보건지소,기타,지원대상\n○ 임신가능성이 없는 가임기 여성으로 풍진항체검사 음성인 군민\n○ 최근...,지원내용\n풍진예방접종 무료지원 ( MMR백신 ) : 1회,영암군보건소 ( ☎061-470-6577 ),NaN,임신/출산/육아,의료지원,(1) 임신가능성이 없는 가임기 여성으로 풍진항체검사 음성인 군민\n(2) 최근 4...,임신가능성이 없는 가임기 여성으로 풍진항체검사 음성인 군민 등,(1) 풍진예방접종 무료지원 ( MMR백신 ) : 1회,(1) 풍진예방접종 무료지원 ( MMR백신 ) : 1회,"정책명은 예비맘 풍진 예방접종지원이고, 지역은 전라남도/영암군이고, 지원대상은 임신..."


In [ ]:
csv_text.head(5)  # 필터링된 결과 상위 5개

In [1]:
from policy_search import find_policies
# 키워드만 → score>0만
df = find_policies(input="2025년 경기임산부 친환경농산물 지원",
                   region="", dob="", categories=[], supports=[], out="dataframe")
print(df.shape, df.head(3))

# 키워드 없음 + 전국 + 카테고리 3개 → 전체데이터에서 OR
df2 = find_policies(input="",
                    region="전국", dob="",
                    categories=["1인가구","고령자","한부모"], supports=[],
                    out="dataframe")
print(df2.shape, df2[['제목','카테고리_분류' if '카테고리_분류' in df2.columns else 'category_label']].head(3))

# # 키워드 없음 + 서울특별시 + 카테고리 1개 → 해당 지역 startswith + OR
# df3 = find_policies(input="",
#                     region="서울특별시", dob="",
#                     categories=["1인가구"], supports=[],
#                     out="dataframe")

# 키워드 없음 + 서울특별시 + 카테고리 1개 → 해당 지역 startswith + OR
df3 = find_policies(input="",
                    region="서울특별시", dob="",
                    categories=["대학(원)생"], supports=[],
                    out="dataframe")

print(df3['지역'].unique()[:5], df3.shape)


c:\Users\hyunj\anaconda3\envs\env1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(1292, 21)     score  orig_index 대상유형         지역                     제목  \
0  0.7957          62  임산부        경기도  2025년 경기임산부 친환경농산물 지원   
1  0.6708          83  임산부    경기도/고양시             임신중 영양제 지원   
2  0.6618         641  임산부  서울특별시/동작구      임산부 친환경농산물 꾸러미 지원   

                                          detail_url  \
0  https://gg24.gg.go.kr/svcreqst/selectSvcReqst....   
1  https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3940...   
2  https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3190...   

                                              신청기간  \
0  2025.05.12 ( 월 ) 09:00 ~ 2025.10.31 ( 금 ) 18:00   
1                                             상시신청   
2                                      공고 및 대상 모집시   

                                                신청방법      접수기관  \
0  ❍ 신청방법 : ( 온라인 ) 경기민원24 ( https://gg24.gg.go.k...   경기도민원24   
1  ○ 방문 신청\n - 보건소 : 관할 보건소 방문\n - 주민센터: 맘편한 임신 서...  주민센터,보건소   
2  임산부 비대면자격검증시스템 온라인 신청 ( 신청 사이트 주소 등 구체적인 내용 동작...      동작구청   

                         

In [2]:
# 키워드 없음 + 서울특별시 + 카테고리 1개 → 해당 지역 startswith + OR
from search.policy_search import find_policies
df3 = find_policies(input="대구에서 진행하는 45세 이상 여성 정책",
                    region="", 
                    dob="",
                    categories=[], supports=[],
                    out="dataframe")

# print(df3['지역'].unique()[:5], df3.shape)
df3.head(5)  # 상위 5개 출력


,score,orig_index,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,...,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약,age_eff_ranges,age_has_rule,검색본문_nat
0,0.605763,492,여성,대구광역시/달성군,오픈마켓 전문셀러 양성사업,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3480...,2024.06.13~2024.08.09,◉ 상시 공고에 따른 신청,한국폴리텍대학 남대구캠퍼스,"서비스(일자리), 기타(교육)",...,NaN,"구직/취업,창업/자영업,취약계층","직접금전지원,교육지원,교통지원,문화예술,서비스",(1) 달성군 거주 여성,(1) 달성군 거주 여성,(1) 창업 훈련내용\n- 직업기초 인성교육\n- 전문셀러광고&운영실전\n- 현장체...,직업기초 인성교육 등,[],False,"정책명은 오픈마켓 전문셀러 양성사업이고, 지역은 대구광역시/달성군이고, 지원대상은 ..."
1,0.600581,506,임산부,대구광역시/서구,임신부 엽산제·철분제 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3430...,상시신청,○ 방문 신청\n - 보건소 : 관할 보건소 모자보건실 방문,보건소,현물,...,NaN,임신/출산/육아,물품지원,(1)관내 임신부,(1)관내 임신부,(1)관내 임신부(임신초기~12주)에게 엽산제 지원\n(2)관내 임신부(16주~ 분...,(1)관내 임신부(임신초기~12주)에게 엽산제 지원\n(2)관내 임신부(16주~ 분...,[],False,"정책명은 임신부 엽산제·철분제 지원이고, 지역은 대구광역시/서구이고, 지원대상은 관..."
2,0.597814,486,임산부,대구광역시/남구,임산부와 영유아를 위한 건강 도서실,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3440...,상시신청,보건소 방문을 통한 도서 열람 및 대여 (신분증 지참),보건소,시설이용,...,NaN,임신/출산/육아,"문화예술,서비스",(1)관내 임산부 및 영유아 부모,(1)관내 임산부 및 영유아 부모,"(1)태교ㆍ임신ㆍ출산ㆍ육아 관련 도서 대여 (1인당 최대 3권, 2주간 대여 가능)","(1)태교ㆍ임신ㆍ출산ㆍ육아 관련 도서 대여 (1인당 최대 3권, 2주간 대여 가능)",[],False,"정책명은 임산부와 영유아를 위한 건강 도서실이고, 지역은 대구광역시/남구이고, 지원..."
3,0.595712,502,한부모,대구광역시/북구,저소득 주민 국민건강보험료 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3450...,상시신청,○ 개인 신청절차 없음\n - 별도의 신청없이 국민건강보험공단에서 대상자 선정,국민건강보험공단,현금(보험),...,NaN,"한부모,고령자,취약계층","직접금전지원,보험료지원",(1)국민건강보험공단 대구북부지사 가입자이면서 보험료의 부과금액이 최저보험료 이하인...,국민건강보험공단 대구북부지사 가입자이면서 보험료의 부과금액이 최저보험료 이하인 세대...,(1)저소득주민 국민건강보험료 지원,(1)저소득주민 국민건강보험료 지원,"[[65, 200]]",True,"정책명은 저소득 주민 국민건강보험료 지원이고, 지역은 대구광역시/북구이고, 지원대상..."
4,0.595530,505,임산부,대구광역시/서구,예비부부(부모) 무료 건강검진 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3430...,상시신청,○ 방문 신청\n - 보건소 : 관할 보건소 모자보건실 방문,보건소,서비스(의료),...,NaN,신혼부부,"의료지원,서비스",(1)관내 예비부부(부모),(1)관내 예비부부(부모),"(1)무료 건강검진(10종) 실시 및 엽산제 지원(여성)\n(2)B형간염, 에이즈,...","(1)무료 건강검진(10종) 실시 및 엽산제 지원(여성)\n(2)B형간염, 에이즈,...",[],False,"정책명은 예비부부(부모) 무료 건강검진 지원이고, 지역은 대구광역시/서구이고, 지원..."


In [ ]:
df3 = find_policies(input="",
                    region="대구", 
                    dob="1981-05-20",
                    categories=[], supports=[],
                    out="dataframe")
df3.head(5)  # 상위 5개 출력

,orig_index,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,...,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약,age_eff_ranges,age_has_rule,검색본문_nat
0,463,한부모,대구광역시,기존 주택 전세 임대,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/O000...,2023.01.01~2023.02.28,○ 방문 신청\n - 주민센터 : 거주지 관할 주민센터에 접수(매년 1~2월 일괄 접수),주민센터,"현금, 현물","지원대상\n○ 1순위\n - 「국민기초생활 보장법」에 의한 생계, 의료급여 수급자\...",...,NaN,"한부모,고령자,취약계층","직접금전지원,주거지원,물품지원","(1) 해당 1순위\n - 「국민기초생활 보장법」에 의한 생계, 의료급여 수급자\n...","「국민기초생활 보장법」에 의한 생계, 의료급여 수급자 등","(1)전세금 지원 : 최대 8,550만원(9,000만원의 95%)","(1)전세금 지원 : 최대 8,550만원(9,000만원의 95%)",[],False,"정책명은 기존 주택 전세 임대이고, 지역은 대구광역시이고, 지원대상은 1순위 - 「..."
1,861,한부모,전국,등유바우처,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1450...,2023.10.16~2023.11.23,ㅇ 방문신청: 주민등록상 거주지 읍·면·동 행정복지센터 방문\nㅇ 직권신청: 담당 ...,주민센터,이용권,지원대상\n○ 기름보일러를 사용하는 생계·의료급여 수급세대 중 한부모가족 또는 소년...,...,NaN,"한부모,임신/출산/육아,취약계층","직접금전지원,이용권,의료지원",(1) 기름보일러를 사용하는 생계·의료급여 수급세대 중 한부모가족 또는 소년소녀가정...,기름보일러를 사용하는 생계·의료급여 수급세대 중 한부모가족 또는 소년소녀가정(가정위...,(1) 기준을 충족하는 세대에 등유바우처(가구당 64.1만원) 지원\n- 기름보일러...,기준을 충족하는 세대에 등유바우처(가구당 64.1만원) 지원,[],False,"정책명은 등유바우처이고, 지역은 전국이고, 지원대상은 기름보일러를 사용하는 생계·의..."
2,513,여성,대전광역시,대전시 한방난임 치료비 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/6300...,2024.02.01~2024.03.31,방문 신청 : 전화상담 후 증빙서류 첨부하여 방문신청\n - 신청처 : (사)대한한...,(사)대한한의사협회 대전광역시지부 042-252-8909,"현금, 서비스(의료)",지원대상\n대전시 6개월이상 거주 1982.1.1.이후 출생 난임여성 / 소득기준 ...,...,NaN,"임신/출산/육아,신혼부부","직접금전지원,의료지원,문화예술,서비스",(1)대전시 6개월이상 거주 1982.1.1.이후 출생 난임여성 / 소득기준 없음\...,대전시 6개월이상 거주 1982.1.1.이후 출생 난임여성,(1)대전시 6개월 이상 거주 1982.1.1.이후 출생 난임여성에게 한방치료비(비...,(1)대전시 6개월 이상 거주 1982.1.1.이후 출생 난임여성에게 한방치료비(비...,"[[0, 200]]",True,"정책명은 대전시 한방난임 치료비 지원이고, 지역은 대전광역시이고, 지원대상은 대전시..."
3,1074,한부모,전라남도/목포시,희망장학금,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/O000...,2024.05.01.~2024.05.31.,○ 방문 신청\n - 주민센터 : 주소지 동 행정복지센터\n -목포어울림도서관 인재육성과,"주민센터,시·군·구청",현금(장학금),"지원대상\n관내 중·고등학생 중 저소득, 다자녀, 다문화, 장애인, 농·어업인, 새...",...,NaN,"한부모,초/중/고등학생,다문화,농/어업인,취약계층","직접금전지원,교육지원,문화예술","(1)관내 중·고등학생 중 저소득, 다자녀, 다문화, 장애인, 농·어업인, 새터민,...","(1)관내 중·고등학생 중 저소득, 다자녀, 다문화, 장애인, 농·어업인, 새터민,...","(1)수혜자에게 일시적으로 제공하는 현금성 장학금\n- 저소득가정, 다자녀가정, 다...",수혜자에게 일시적으로 제공하는 현금성 장학금,[],False,"정책명은 희망장학금이고, 지역은 전라남도/목포시이고, 지원대상은 관내 중·고등학생 ..."
4,492,여성,대구광역시/달성군,오픈마켓 전문셀러 양성사업,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/3480...,2024.06.13~2024.08.09,◉ 상시 공고에 따른 신청,한국폴리텍대학 남대구캠퍼스,"서비스(일자리), 기타(교육)",지원대상\n◉ 달성군 거주 여성,...,NaN,"구직/취업,창업/자영업,취약계층","직접금전지원,교육지원,교통지원,문화예술,서비스",(1) 달성군 거주 여성,(1) 달성군 거주 여성,(1) 창업 훈련내용\n- 직업기초 인성교육\n- 전문셀러광고&운영실전\n- 현장체...,직업기초 인성교육 등,[],False,"정책명은 오픈마켓 전문셀러 양성사업이고, 지역은 대구광역시/달성군이고, 지원대상은 ..."


In [ ]:
"C:\Users\hyunj\Desktop\Downloads\여성맞춤정책_요약_2차_결과_정리.xlsx"
"C:\Users\hyunj\Desktop\new_fold\women_search_UI\UI\여성맞춤정책_요약_2차_결과_정리.xlsx"

In [5]:
# strict_diff_policy_columns_autodetect.py
# - 네 열(지원내용_원문/요약, 지원대상_원문/요약)을 '자동 탐지' 후
#   행 대 행으로 "한 글자라도 다르면" 잡아 엑셀로 저장
from pathlib import Path
import pandas as pd
import re, math
import unicodedata as ud
from collections import defaultdict

FILE_A = r"C:\Users\hyunj\Desktop\Downloads\여성맞춤정책_요약_2차_결과_정리.xlsx"
FILE_B = r"C:\Users\hyunj\Desktop\new_fold\women_search_UI\UI\여성맞춤정책_요약_2차_결과_정리.xlsx"
OUT_XLSX = Path(FILE_B).with_name("정책_열별_완전일치_검사결과.xlsx")

# ─────────────────────────────────────────────────────────────
# 1) 정규화 & 토큰 매칭
# ─────────────────────────────────────────────────────────────
def norm(s: str) -> str:
    if s is None or (isinstance(s, float) and math.isnan(s)):
        return ""
    s = ud.normalize("NFKC", str(s))
    s = s.strip()
    # 공백/밑줄/하이픈 제거, 괄호/기호 제거
    s = re.sub(r"[\s_\-]+", "", s)
    s = re.sub(r"[()\[\]{}·•・·,./\\:;!?]", "", s)
    return s

# 동의어/키워드 (요약/원문 그룹 구분)
TOKENS = {
    "원문": ["원문", "본문", "상세", "원본", "풀텍스트", "상세내용"],
    "요약": ["요약", "요지", "개요", "핵심", "요약문"],
    "지원내용": ["지원내용", "내용", "지원내역", "사업내용", "정책내용"],
    "지원대상": ["지원대상", "대상", "수혜대상", "신청대상", "적격대상"],
}

def score_col(col_norm: str, group: str) -> int:
    """
    group ∈ {
      '지원내용_원문','지원내용_요약','지원대상_원문','지원대상_요약'
    }
    점수가 높을수록 해당 타깃에 적합
    """
    base, kind = group.split("_")  # base: 지원내용/지원대상, kind: 원문/요약
    score = 0
    # 기본 토큰
    for t in TOKENS[base]:
        if t in col_norm:
            score += 3
    # 원문/요약 구분 토큰
    for t in TOKENS[kind]:
        if t in col_norm:
            score += 5
    # 상충 토큰(예: '원문' 타깃인데 '요약' 포함) 패널티
    other = "원문" if kind == "요약" else "요약"
    if any(t in col_norm for t in TOKENS[other]):
        score -= 4
    # 완전 일치 보너스
    canonical = norm(group)
    if col_norm == canonical:
        score += 6
    return score

TARGETS = ["지원내용_원문","지원내용_요약","지원대상_원문","지원대상_요약"]

def pick_columns(df: pd.DataFrame) -> dict:
    """
    데이터프레임에서 TARGETS에 해당하는 열을 자동 선택.
    반환: {타깃열명 -> 실제열명 or None}
    """
    cols = list(df.columns)
    norms = {c: norm(c) for c in cols}
    mapping = {}
    used = set()
    for tgt in TARGETS:
        # 모든 컬럼에 대해 스코어링
        scored = []
        for c in cols:
            if c in used:
                continue
            s = score_col(norms[c], tgt)
            # 최소 스코어 기준(경계 낮춤)
            if s >= 4:
                scored.append((s, len(norms[c]), c))
        if scored:
            # 높은 점수, 다음으로 문자열 길이 짧은 것(불필요 수식어 적은 것) 우선
            scored.sort(key=lambda x: (-x[0], x[1]))
            best = scored[0][2]
            mapping[tgt] = best
            used.add(best)
        else:
            mapping[tgt] = None
    return mapping

def read_best_sheet(xlsx_path: str) -> tuple[pd.DataFrame, dict, str]:
    """
    모든 시트를 읽어, 매칭된 타깃 열 수가 가장 많은 시트를 선택.
    반환: (df, 매핑, 시트명)
    """
    book = pd.read_excel(xlsx_path, sheet_name=None, dtype=object, keep_default_na=False, engine="openpyxl")
    best = None
    best_map = None
    best_sheet = None
    best_hits = -1
    for sheet, df in book.items():
        mapping = pick_columns(df)
        hits = sum(1 for k,v in mapping.items() if v is not None)
        if hits > best_hits:
            best_hits = hits
            best = df
            best_map = mapping
            best_sheet = sheet
    return best, best_map, best_sheet

# ─────────────────────────────────────────────────────────────
# 2) 완전 동일 비교 유틸
# ─────────────────────────────────────────────────────────────
def isna(x):
    return x is None or (isinstance(x, float) and math.isnan(x)) or pd.isna(x)

def cells_equal(a, b):
    if isna(a) and isna(b):
        return True
    return a == b

def first_diff_index(a, b):
    sa = "" if isna(a) else str(a)
    sb = "" if isna(b) else str(b)
    n = min(len(sa), len(sb))
    for i in range(n):
        if sa[i] != sb[i]:
            return i
    return -1 if len(sa) == len(sb) else n

def context(s, pos, ctx=15):
    if pos < 0:
        return ""
    s = "" if isna(s) else str(s)
    start = max(0, pos - ctx)
    end   = min(len(s), pos + ctx)
    return s[start:end]

# ─────────────────────────────────────────────────────────────
# 3) 메인: 시트 자동선택 + 열 자동매핑 + 행대행 비교
# ─────────────────────────────────────────────────────────────
def main():
    dfA, mapA, sheetA = read_best_sheet(FILE_A)
    dfB, mapB, sheetB = read_best_sheet(FILE_B)

    # 행 수 맞추기(인덱스 기준)
    n = min(len(dfA), len(dfB))
    if n == 0:
        raise ValueError("두 파일 중 하나가 비어있습니다.")
    A = dfA.iloc[:n].reset_index(drop=True)
    B = dfB.iloc[:n].reset_index(drop=True)

    logs = []
    logs.append({"파일":"A","선택시트":sheetA})
    logs.append({"파일":"B","선택시트":sheetB})
    for t in TARGETS:
        logs.append({"파일":"A","타깃열":t,"매핑된열":mapA.get(t)})
    for t in TARGETS:
        logs.append({"파일":"B","타깃열":t,"매핑된열":mapB.get(t)})

    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as xw:
        # 비교 실행
        summary = []
        all_rows = []

        for tgt in TARGETS:
            colA = mapA.get(tgt)
            colB = mapB.get(tgt)

            # 둘 중 하나라도 없으면 빈 시트 생성
            if not colA or not colB or colA not in A.columns or colB not in B.columns:
                empty = pd.DataFrame(columns=["행번호(0기준)","열명","A값","B값","첫불일치_위치(0기준)","A_문맥","B_문맥","A_길이","B_길이"])
                empty.to_excel(xw, sheet_name=f"{tgt}_diff", index=False)
                summary.append([tgt, 0, "열 없음(매핑 실패)"])
                continue

            diffs = []
            for i in range(n):
                va = A.at[i, colA]
                vb = B.at[i, colB]
                if not cells_equal(va, vb):
                    pos = first_diff_index(va, vb)
                    diffs.append({
                        "행번호(0기준)": i,
                        "열명": tgt,
                        "A열명": colA,
                        "B열명": colB,
                        "A값": va,
                        "B값": vb,
                        "첫불일치_위치(0기준)": pos,
                        "A_문맥": context(va, pos),
                        "B_문맥": context(vb, pos),
                        "A_길이": 0 if isna(va) else len(str(va)),
                        "B_길이": 0 if isna(vb) else len(str(vb)),
                    })

            diff_df = pd.DataFrame(diffs, columns=[
                "행번호(0기준)","열명","A열명","B열명","A값","B값",
                "첫불일치_위치(0기준)","A_문맥","B_문맥","A_길이","B_길이"
            ])
            diff_df.to_excel(xw, sheet_name=f"{tgt}_diff", index=False)
            summary.append([tgt, len(diff_df), "OK" if len(diff_df) or True else "OK"])
            if len(diff_df):
                all_rows.append(diff_df)

        # 요약
        summary_df = pd.DataFrame(summary, columns=["타깃열","다른_행_개수","상태"])
        summary_df.to_excel(xw, sheet_name="요약", index=False)

        # 로그(시트/열 매핑 내역 + 원본 열 이름 목록)
        log_df = pd.DataFrame(logs)
        # 원본 열 목록
        colsA = pd.DataFrame({"파일":"A","열이름":list(A.columns)})
        colsB = pd.DataFrame({"파일":"B","열이름":list(B.columns)})
        log_all = pd.concat([log_df, pd.DataFrame([{}]), colsA, colsB], ignore_index=True)
        log_all.to_excel(xw, sheet_name="로그", index=False)

        if all_rows:
            merged_all = pd.concat(all_rows, ignore_index=True)
            merged_all.to_excel(xw, sheet_name="모든열_diff", index=False)

    print(f"[완료] {OUT_XLSX} 저장")
    print("시트: 각 타깃별 *_diff + 요약 + 로그 (+ 모든열_diff)")

if __name__ == "__main__":
    main()


[완료] C:\Users\hyunj\Desktop\new_fold\women_search_UI\UI\정책_열별_완전일치_검사결과.xlsx 저장
시트: 각 타깃별 *_diff + 요약 + 로그 (+ 모든열_diff)
